# Name: Nora Mohamed Ebrahim
# ID: 4221310
# Subgroup: A5 - AI

In [6]:
# Define functions

# Create vocabulary from words
def build_vocab(words):
    vocab = []
    for word in words:
        if word not in vocab:
            vocab.append(word)
    word_to_index = {}
    index_to_word = {}
    for i in range(len(vocab)):
        word_to_index[vocab[i]] = i
        index_to_word[i] = vocab[i]
    return vocab, word_to_index, index_to_word

# Simple exponential function (for softmax)
def exp(x):
    n = 20
    result = 1.0
    power = 1.0
    factorial = 1.0
    for i in range(1, n):
        power = power * x
        factorial = factorial * i
        result = result + power / factorial
    return result

# Softmax function
def softmax(logits):
    exps = []
    for i in range(len(logits)):
        exps.append(exp(logits[i]))
    sum_exps = 0.0
    for val in exps:
        sum_exps += val
    softmax_out = []
    for val in exps:
        softmax_out.append(val / sum_exps)
    return softmax_out

# Tanh activation function
def tanh(vector):
    out = []
    for val in vector:
        e_pos = exp(val)
        e_neg = exp(-val)
        t = (e_pos - e_neg) / (e_pos + e_neg)
        out.append(t)
    return out

# Initialize RNN weights
def initialize_weights(vocab_size, hidden_size):
    Wxh = []
    for i in range(vocab_size):
        row = []
        for j in range(hidden_size):
            row.append(0.01 * (i + j + 1))
        Wxh.append(row)

    Whh = []
    for i in range(hidden_size):
        row = []
        for j in range(hidden_size):
            row.append(0.01 * (i - j + 2))
        Whh.append(row)

    Why = []
    for i in range(hidden_size):
        row = []
        for j in range(vocab_size):
            row.append(0.01 * (i + j + 2))
        Why.append(row)

    bh = []
    for i in range(hidden_size):
        bh.append(0.0)

    by = []
    for i in range(vocab_size):
        by.append(0.0)

    return Wxh, Whh, Why, bh, by

# Forward pass through RNN
def forward_pass(inputs, Wxh, Whh, Why, bh, by, vocab_size, hidden_size):
    hs = []
    h_prev = []
    for i in range(hidden_size):
        h_prev.append(0.0)
    hs.append(h_prev)

    for t in range(len(inputs)):
        x = []
        for i in range(vocab_size):
            if i == inputs[t]:
                x.append(1.0)
            else:
                x.append(0.0)

        h_raw = []
        for j in range(hidden_size):
            sum_x = 0.0
            for i in range(vocab_size):
                sum_x += x[i] * Wxh[i][j]
            sum_h = 0.0
            for i in range(hidden_size):
                sum_h += hs[-1][i] * Whh[i][j]
            h_raw.append(sum_x + sum_h + bh[j])

        h = tanh(h_raw)
        hs.append(h)

    y_raw = []
    for j in range(vocab_size):
        sum_y = 0.0
        for i in range(hidden_size):
            sum_y += hs[-1][i] * Why[i][j]
        y_raw.append(sum_y + by[j])

    y = softmax(y_raw)
    return y

# Predict the next word
def predict(words):
    vocab, word_to_index, index_to_word = build_vocab(words)
    vocab_size = len(vocab)
    hidden_size = 5

    Wxh, Whh, Why, bh, by = initialize_weights(vocab_size, hidden_size)

    inputs = [word_to_index[words[0]], word_to_index[words[1]], word_to_index[words[2]]]

    output_probs = forward_pass(inputs, Wxh, Whh, Why, bh, by, vocab_size, hidden_size)

    predicted_index = 0
    max_val = output_probs[0]
    for i in range(len(output_probs)):
        if output_probs[i] > max_val:
            max_val = output_probs[i]
            predicted_index = i

    return index_to_word[predicted_index]

In [7]:
my_words = ["I", "love", "machine", "learning"]
print("Predicted 4th word:", predict(my_words))

Predicted 4th word: learning
